In [310]:
# standard imports
import numpy as np
import os
import pandas as pd
import pickle
import yaml

from datetime import timedelta
from pathlib import Path

# bokeh
from bokeh.io import output_notebook, export_png, export_svgs
from bokeh.layouts import gridplot
from bokeh.palettes import Blues3 as palette
from bokeh.models import HoverTool, Title, FactorRange, LinearAxis, Legend, Band, Range1d, LabelSet, Span
from bokeh.palettes import Blues
from bokeh.plotting import figure, show, output_file, ColumnDataSource
from bokeh.transform import factor_cmap
from bokeh.sampledata.us_counties import data as counties
output_notebook()

# lib
import sys
sys.path.append('../')
from metrics import compute_metrics, _compute_metrics

Loading BokehJS ...

In [404]:
def load_backfill(job, basedir='/checkpoint/maxn/covid19/forecasts'):
    """collect all forcasts from job dir"""
    jobdir = os.path.join(basedir, job)
    forecasts = {}
    configs = []
    for path in Path(jobdir).rglob('final_model_piv.csv'):
        date = str(path).split('/')[7]
        assert date.startswith('sweep_'), date
        date = date[6:]
        forecasts[date] = path
        job = '/'.join(str(path).split('/')[:-1])
        cfg = job + '/ar.yml'
        cfg = yaml.load(open(cfg), Loader=yaml.FullLoader)['train']
        cfg['date'] = date
        cfg['job'] = job
        configs.append(cfg)
    configs = pd.DataFrame(configs)
    configs.set_index('date', inplace=True)
    return forecasts, configs
        
def load_predictions(path):
    df = pd.read_csv(path, index_col='date', parse_dates=['date'])
    return df

def plot_metric(mets, other, title, metric):
    source = ColumnDataSource(mets)
    p = figure(
        x_axis_type='datetime', 
        plot_height=350, 
        plot_width=450, 
        title=f"Forecast Quality {title}", 
        tools="save,hover",
        x_axis_label='Day', 
        y_axis_label=metric,
     )
    p.extra_y_ranges = {"counts": Range1d(start=mets['counts'].min(), end=mets['counts'].max())}
    p.add_layout(LinearAxis(y_range_name="counts", axis_label='Deaths'), 'right')
    
    l_ar = p.line(x='day', y='AR', source=source, line_width=3, color='black', legend_label='FAIR-AR')
    l_na = p.line(x='day', y='Naive', source=source, line_width=3, color='#009ed7', legend_label='Naive', line_dash='dotted')
    #l_ot = p.line(x='day', y=other, source=source, line_width=3, color='#009ed7', legend_label=other)
    p.line(x='day', y='counts', source=source, line_width=1, color="LightGray", line_alpha=0.2, y_range_name='counts')
    band = Band(base='day', upper='counts', source=source, level='underlay', fill_alpha=0.5, fill_color='LightGray', y_range_name='counts')
    p.add_layout(band)
    p.y_range.renderers = [l_ar, l_na]

    p.legend.location = 'top_left'
    p.output_backend = 'svg'
    p.background_fill_color = 'white'
    p.border_fill_color = 'white'
    p.outline_line_color = 'white'
    p.title.text_font = 'Montserrat'
    p.title.text_font_style = 'normal'
    p.title.text_color = '#677b8c'

    return p

def plot_metric_for_dates(fs, dates, other, model, metric='MAE', state=None):
    ps = []
    for date in dates:
        #df_other = load_predictions(f'/checkpoint/mattle/covid19/csvs/deaths/{model}/counts_{date}.csv').iloc[1:]
        df_ar = pd.read_csv(fs[date], index_col='date', parse_dates=['date'])
        df_gt = df_states
        if state is not None:
            #df_other = df_other[state].to_frame()
            df_ar = df_ar[state].to_frame()
            df_gt = df_gt[state].to_frame()

        #met_other = _compute_metrics(df_gt, df_other)
        met_ar = _compute_metrics(df_gt, df_ar)
        display(met_ar)
        source = pd.DataFrame({
            'Naive': met_ar.loc[f'{metric}_NAIVE'],
            # other: met_other.loc[metric], 
            'AR': met_ar.loc[metric],
        })
        source.index.set_names('day', inplace=True)
        source['counts'] = df_gt.loc[source.index].sum(axis=1)

        region = 'US' if state is None else state
        p = plot_metric(source, other, f'{region} {date}', metric)
        ps.append(p)
    return ps

def plot_accuracy(mets, plevel, exclude={}, title='Confirmed Cases'):
    mets = mets[~mets['date'].isin(exclude)]
    mets['date'] = pd.to_datetime(mets['date'])
    mets = mets.sort_values(by='date')
    x = list(zip(
        map(lambda x: x.strftime('%m/%d'), mets['date']), 
        map(str, mets['days'])
    ))
    source = ColumnDataSource(data=dict(x=x, acc=mets['acc'].round(2)))

    p = figure(
        x_range=FactorRange(*x), 
        plot_height=250,
        plot_width=700,
        title=f"Forecast Accuracy Spain, {title} - Confidence Level {plevel}",
        x_axis_label="Number of days forecasted on date",
        y_axis_label="Accuracy",
        tools="save",
        y_range=Range1d(start=0,end=1.1),
    )

    p.vbar(x='x', top='acc', width=0.85, source=source, line_color='black',
        fill_color=factor_cmap('x', palette=palette, factors=['7', '14', '21'], start=1, end=2))
    labels = LabelSet(x='x', y='acc', text='acc', level='glyph', x_offset=-11.5, y_offset=0, source=source, 
        text_font_size='0.85em', render_mode='canvas')
    p.add_layout(labels)

    p.y_range.start = 0
    p.x_range.range_padding = 0.1
    p.xaxis.major_label_orientation = 1
    p.xgrid.grid_line_color = None
    p.output_backend = 'svg'
    return p

### Progression in the US

In [405]:
# Load ground truth data
df_states = pd.read_csv('../data/spain/data_deaths.csv', index_col='region')
df_states = df_states.transpose()
df_states.index.set_names(['date'], inplace=True)
df_states.index = pd.to_datetime(df_states.index)
df_states

region,Spain_Andalucía,Spain_Aragón,Spain_Asturias,Spain_Canarias,Spain_Cantabria,Spain_Castilla y León,Spain_Castilla-La Mancha,Spain_Cataluña,Spain_Ceuta,Spain_Extremadura,Spain_Galicia,Spain_Islas Baleares,Spain_La Rioja,Spain_Madrid,Spain_Melilla,Spain_Murcia,Spain_Navarra,Spain_País Vasco,Spain_Valenciana
date,,,,,,,,,,,,,,,,,,,
2020-03-08,0.0,3.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,16.0,0.0,0.0,0.0,6.0,1.0
2020-03-09,0.0,4.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0,21.0,0.0,0.0,0.0,6.0,1.0
2020-03-10,0.0,4.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,1.0,0.0,0.0,2.0,31.0,0.0,0.0,0.0,6.0,1.0
2020-03-11,0.0,6.0,1.0,0.0,0.0,0.0,1.0,6.0,0.0,1.0,0.0,1.0,2.0,56.0,0.0,0.0,0.0,11.0,1.0
2020-03-12,0.0,7.0,1.0,0.0,0.0,0.0,5.0,9.0,0.0,1.0,0.0,1.0,3.0,81.0,0.0,0.0,0.0,14.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-05-13,1336.0,836.0,308.0,151.0,205.0,1925.0,2852.0,5823.0,4.0,492.0,603.0,216.0,348.0,8779.0,2.0,142.0,498.0,1454.0,1349.0
2020-05-14,1344.0,837.0,310.0,151.0,206.0,1930.0,2870.0,5882.0,4.0,494.0,603.0,216.0,348.0,8809.0,2.0,143.0,500.0,1454.0,1358.0
2020-05-15,1355.0,838.0,313.0,151.0,206.0,1940.0,2883.0,5915.0,4.0,497.0,604.0,216.0,348.0,8826.0,2.0,144.0,501.0,1455.0,1365.0


In [406]:
# plot deaths over time 
print(df_states.shape)
source = ColumnDataSource(df_states)
p = figure(
    x_axis_type='datetime', 
    plot_height=350, 
    plot_width=500, 
    title=f"Morbidity per Region", 
    tools="save",
    x_axis_label='Day', 
    y_axis_label='Deaths',
)
states = df_states.columns
for state in states:
    p.line(x='date', y=state, source=source, line_width=2, color='#009ed7')
p.output_backend = 'svg'
show(p)

(71, 19)


### Load Backfill and Configs

In [414]:
# deaths
job = "es/2020_05_18_19_34"
job = "es/2020_05_18_19_54"
job = "es/2020_05_18_21_11"
job = "es/2020_05_22_12_54"
job = "es/2020_05_22_13_21"
job = "es/2020_05_22_13_44"
job = "es/2020_05_25_12_45"

# cases
#job = "es/2020_05_19_15_47"
#job = "es/2020_05_19_20_32"
#job = "es/2020_05_20_21_31"
fs, cfgs = load_backfill(job) 
cfgs.drop(columns=['fdat', 'fpop', 'job'])

,decay,loss,lr,momentum,niters,t0,test_on,weight_decay,window
date,,,,,,,,,
2020-04-22,latent2_1,nb,0.001,0.99,30000,5,21,0.5,10
2020-04-29,latent2_1,nb,0.001,0.99,30000,5,21,0.5,15
2020-05-05,latent2_1,nb,0.001,0.99,30000,5,21,0.5,10
2020-04-19,latent2_1,nb,0.001,0.99,30000,5,21,0.0,10
2020-04-26,latent2_1,nb,0.001,0.99,30000,5,21,0.5,15
2020-05-01,latent2_1,nb,0.001,0.99,30000,5,21,0.2,10


In [415]:
from collections import defaultdict as ddict
pi_multipliers = {
    .99: 2.58,
    .95: 1.96,
    .80: 1.28, 
}
ps = []
mets = []
pivs = ddict(list)
plevel = .99
def select_piv(df, interval):
    _df = df.drop(columns=['piv'])
    return _df[df["piv"] == interval].set_index('date')
for date in cfgs.index:
    jobdir = cfgs.loc[date]['job']
    df_piv = pd.read_csv(f'{jobdir}/../forecasts/piv_best_mae.csv', parse_dates=['date'])
    lower = select_piv(df_piv, str(np.round(1 - plevel, 2)))
    upper = select_piv(df_piv, str(plevel))
    mean = select_piv(df_piv, "mean")

    ix = np.intersect1d(mean.index, df_states.index)
    df_gt = df_states.loc[ix]
    mean = mean.loc[ix]
    lower = lower.loc[ix]
    upper = upper.loc[ix]

    n = len(df_gt)
    def plot_prediction_interval(county, p, start=0):
        x = range(1, n + 1)
        df = pd.DataFrame({
            'x': x,
            'y': df_gt[county].values
        })
        source = ColumnDataSource(df)
        p.line(x='x', y='y', source=source, color='#009ed7', line_width=2, legend_label='Ground Truth')
    
        y = mean[county].values
        df = pd.DataFrame({
            'x': x, 
            'y': y,
            'lower': lower[county].values,
            'upper': upper[county].values,
        })
        source = ColumnDataSource(df)
        p.line(x='x', y='y', source=source, line_width=2, color='black', alpha=0.5, line_dash='dotted', legend_label='Prediction')
        band = Band(base='x', lower='lower', upper='upper', source=source, level='underlay',
                fill_alpha=0.8, line_width=1, line_color='LightGray', fill_color='LightGray')
        p.legend.location='bottom_left'
        p.add_layout(band)
        #for _d in [7, 14, 21]:
        #    d = Span(location=_d, dimension='height', line_color='Teal', line_alpha=0.1, line_width=2)
        #    d.level = "underlay"
        #    p.add_layout(d)
        p.output_backend = 'svg'
        return p


    p = figure(title=f"Prediction Intervals {date[-5:]}, {len(df_mean)} days", 
               plot_width=300, plot_height=300, tools='save,hover',
              x_axis_label="Forecast days", y_axis_label="Cases")
    regions = df_states.columns
    for i, region in enumerate(regions):
        p = plot_prediction_interval(region, p)
    ps.append(p)
    
    for d in [6, 13, 20]:
        if d >= len(df_gt):
            continue
        z = np.logical_and(df_gt.iloc[d] < upper.iloc[d], df_gt.iloc[d] > lower.iloc[d])
        acc = sum(z) / len(z)
        mets.append((date, d + 1, acc))
        pivs[date].append((df_gt.iloc[d], mean.iloc[d], lower.iloc[d], upper.iloc[d]))

# plot accuracies        
mets = pd.DataFrame(mets, columns=['date', 'days', 'acc'])
p = plot_accuracy(mets, plevel, {'2020-04-19'}, 'Morbidity')
show(p)

# plot prediciton intervals
# grid = gridplot(ps, ncols=2, plot_width=430)
grid = gridplot([ps[0], ps[-2], ps[1]], ncols=3)
show(grid)
_ = export_svgs(grid, filename='/tmp/piv_esp.svg')

/private/home/maxn/.conda/envs/timelord/lib/python3.7/site-packages/ipykernel_launcher.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [367]:
for date, ts in pivs.items():
    _dfs = []
    for t in ts:
        _df = pd.concat(t, axis=1)
        c = _df.columns
        t = c[0].strftime('%m/%d')
        _df.columns = [f'{t} gt', f'{t} mean', f'{t} lower', f'{t} upper']
        _dfs.append(_df.round(2))
    _dfs = pd.concat(_dfs, axis=1)
    _dfs.to_csv(f'/tmp/piv_spain_{date}.csv')
    pivs[date] = _dfs

In [375]:
pivs['2020-04-26'].sum(axis=0).to_frame()

,0
04/27 gt,214007.00
04/27 mean,213923.40
04/27 lower,211794.50
04/27 upper,216052.37
05/03 gt,220111.00
05/03 mean,220309.00
05/03 lower,214350.39
05/03 upper,226267.59
05/10 gt,227988.00
05/10 mean,224470.70


### IHME

Compare our forecasts to published data by [IHME](https://covid-19.bsvgateway.org/)

In [365]:
dates_los_alamos = [
    '2020-04-22',
    '2020-04-29',
    '2020-05-05',
    '2020-05-12',
    "2020-04-19",
    '2020-04-26',
    '2020-05-01'
]
ps = plot_metric_for_dates(fs, dates_los_alamos, 'IHME', 'IHME', 'MAE', None)
grid = gridplot(ps, ncols=2, plot_width=430)
show(grid)
_ = export_svgs(grid, filename='/tmp/esp.svg')

,2020-04-23,2020-04-24,2020-04-25,2020-04-26,2020-04-27,2020-04-28,2020-04-29,2020-04-30,2020-05-01,2020-05-02,...,2020-05-04,2020-05-05,2020-05-06,2020-05-07,2020-05-08,2020-05-09,2020-05-10,2020-05-11,2020-05-12,2020-05-13
Measure,,,,,,,,,,,,,,,,,,,,,
RMSE,29.656022,102.420835,143.826829,240.061797,281.372826,347.764333,376.448847,420.209140,483.140555,521.871068,...,594.288989,633.695117,689.396789,783.441533,829.045291,866.902484,1341.004840,1408.526031,1441.734320,1453.043453
MAE,21.188133,64.007575,82.810741,117.654800,148.759994,185.565345,203.503301,225.368245,255.946115,273.080778,...,311.475384,331.903376,359.144551,396.553409,419.673997,437.022060,574.010100,597.560412,613.450728,625.823057
MAPE,0.003471,0.006930,0.009684,0.011838,0.014971,0.016345,0.017224,0.018399,0.020166,0.021892,...,0.024570,0.023575,0.024452,0.025491,0.026767,0.027404,0.029524,0.031666,0.031872,0.032947
RMSE_NAIVE,117.249621,212.062801,372.483203,479.392269,618.065062,762.604885,970.705115,1159.440473,1337.462405,1550.022003,...,1998.888573,2232.392132,2450.187802,2620.376158,2857.934081,3104.648950,2914.681927,3130.940980,3377.834817,3649.066413
MAE_NAIVE,49.263158,84.842105,146.157895,196.631579,254.421053,322.736842,412.210526,491.052632,575.052632,671.578947,...,876.578947,977.052632,1074.368421,1159.842105,1264.684211,1372.473684,1361.894737,1465.736842,1575.842105,1693.789474
MAE_MASE,0.430101,0.754432,0.566584,0.598351,0.584700,0.574974,0.493688,0.458949,0.445083,0.406625,...,0.355331,0.339699,0.334284,0.341903,0.331841,0.318419,0.421479,0.407686,0.389284,0.369481
RMSE_MASE,0.252931,0.482974,0.386130,0.500763,0.455248,0.456022,0.387810,0.362424,0.361237,0.336686,...,0.297310,0.283864,0.281365,0.298981,0.290086,0.279227,0.460086,0.449873,0.426822,0.398196


,2020-04-30 00:00:00,2020-05-01 00:00:00,2020-05-02 00:00:00,2020-05-03 00:00:00,2020-05-04 00:00:00,2020-05-05 00:00:00,2020-05-06 00:00:00,2020-05-07 00:00:00,2020-05-08 00:00:00,2020-05-09 00:00:00,2020-05-10 00:00:00,2020-05-11 00:00:00,2020-05-12 00:00:00,2020-05-13 00:00:00,2020-05-14 00:00:00,2020-05-15 00:00:00,2020-05-16 00:00:00,2020-05-17 00:00:00
Measure,,,,,,,,,,,,,,,,,,
RMSE,40.350685,39.970344,86.680966,165.928260,173.956716,256.218863,291.736554,272.472578,338.012583,392.723667,198.908999,181.499579,159.774214,172.487657,204.434356,242.410245,273.655186,315.440874
MAE,24.880943,27.669282,48.117485,72.652876,82.436078,108.215686,129.798603,133.954874,159.879506,176.135728,123.007262,121.152414,111.138585,119.821139,136.332216,156.928096,166.288601,181.155430
MAPE,0.002582,0.003669,0.005696,0.007946,0.009516,0.012218,0.013194,0.013598,0.014866,0.015485,0.014561,0.016557,0.016633,0.018423,0.018705,0.019656,0.020111,0.020796
RMSE_NAIVE,47.885829,121.466045,180.827921,220.174333,252.584223,262.035554,290.007350,353.040403,384.861356,437.180861,848.086453,894.756686,904.767461,902.207818,935.424418,964.707346,1009.001330,1062.801487
MAE_NAIVE,30.000000,62.631579,91.789474,117.052632,139.842105,150.105263,167.210526,200.789474,220.157895,243.631579,406.526316,443.000000,463.315789,478.947368,503.684211,529.631579,554.315789,585.631579
MAE_MASE,0.829365,0.441778,0.524216,0.620686,0.589494,0.720932,0.776259,0.667141,0.726204,0.722959,0.302581,0.273482,0.239877,0.250176,0.270670,0.296297,0.299989,0.309333
RMSE_MASE,0.842644,0.329066,0.479356,0.753622,0.688708,0.977802,1.005963,0.771789,0.878271,0.898309,0.234539,0.202848,0.176591,0.191184,0.218547,0.251279,0.271214,0.296801


,2020-05-06 00:00:00,2020-05-07 00:00:00,2020-05-08 00:00:00,2020-05-09 00:00:00,2020-05-10 00:00:00,2020-05-11 00:00:00,2020-05-12 00:00:00,2020-05-13 00:00:00,2020-05-14 00:00:00,2020-05-15 00:00:00,2020-05-16 00:00:00,2020-05-17 00:00:00
Measure,,,,,,,,,,,,
RMSE,30.578554,45.371742,64.870062,89.061528,457.953733,448.118658,400.985126,337.130393,322.751460,289.291633,262.363064,226.747916
MAE,16.332623,23.899317,38.336570,50.358325,137.566779,142.557189,136.425658,127.826440,133.332496,133.995183,125.538085,118.295666
MAPE,0.001459,0.002512,0.003540,0.004411,0.006670,0.008345,0.008468,0.009712,0.010416,0.011861,0.011362,0.011946
RMSE_NAIVE,29.388505,120.512720,140.240770,184.913778,737.306371,786.485621,792.049706,775.255272,808.192818,824.365753,856.660007,883.408679
MAE_NAIVE,14.000000,41.210526,53.473684,75.842105,225.315789,251.105263,264.842105,273.473684,289.947368,306.578947,324.052632,347.000000
MAE_MASE,1.166616,0.579932,0.716924,0.663989,0.610551,0.567719,0.515121,0.467418,0.459851,0.437066,0.387400,0.340910
RMSE_MASE,1.040494,0.376489,0.462562,0.481638,0.621117,0.569773,0.506263,0.434864,0.399350,0.350926,0.306263,0.256674


,2020-05-13 00:00:00,2020-05-14 00:00:00,2020-05-15 00:00:00,2020-05-16 00:00:00,2020-05-17 00:00:00
Measure,,,,,
RMSE,92.114674,125.118333,170.384538,222.752484,289.062373
MAE,29.221709,47.037618,64.530899,83.143805,100.022967
MAPE,0.002148,0.002994,0.004298,0.005776,0.006077
RMSE_NAIVE,29.005444,23.320422,51.926061,95.062030,139.638631
MAE_NAIVE,13.000000,16.157895,31.578947,47.631579,61.368421
MAE_MASE,2.247824,2.911123,2.043478,1.745561,1.629877
RMSE_MASE,3.175772,5.365183,3.281291,2.343233,2.070075


,2020-04-20,2020-04-21,2020-04-22,2020-04-23,2020-04-24,2020-04-25,2020-04-26,2020-04-27,2020-04-28,2020-04-29,...,2020-05-01,2020-05-02,2020-05-03,2020-05-04,2020-05-05,2020-05-06,2020-05-07,2020-05-08,2020-05-09,2020-05-10
Measure,,,,,,,,,,,,,,,,,,,,,
RMSE,204.451330,248.414043,387.392697,417.538745,463.916285,471.970091,535.916760,585.025647,638.675160,646.970888,...,692.525332,689.046491,662.471255,709.732699,728.911011,761.051569,813.416448,827.577253,820.450443,1150.014402
MAE,82.237271,122.203920,185.784325,209.848172,244.710812,256.642852,283.433819,310.741260,334.183436,345.466688,...,376.360575,378.373205,365.087838,383.914086,389.008158,403.457032,429.009906,437.205264,440.990132,563.930586
MAPE,0.005072,0.008407,0.012045,0.014857,0.018719,0.021776,0.023441,0.026520,0.027351,0.028148,...,0.030565,0.031707,0.032161,0.032730,0.029341,0.029963,0.030625,0.031895,0.032601,0.034995
RMSE_NAIVE,106.827702,210.362469,329.301733,442.430786,585.902138,710.130900,760.683107,967.326511,1141.762419,1355.416017,...,1682.330824,1858.890770,2089.222784,2279.664841,2525.426582,2745.348832,2902.897119,3135.508840,3358.825202,3046.374109
MAE_NAIVE,39.842105,82.368421,137.631579,196.789474,276.052632,330.105263,346.473684,415.736842,469.526316,548.368421,...,668.789474,726.263158,789.842105,856.736842,937.631579,1010.000000,1067.157895,1140.526316,1206.105263,1141.315789
MAE_MASE,2.064079,1.483626,1.349867,1.066359,0.886464,0.777458,0.818053,0.747447,0.711746,0.629990,...,0.562749,0.520986,0.462229,0.448112,0.414884,0.399462,0.402012,0.383336,0.365632,0.494106
RMSE_MASE,1.913842,1.180886,1.176406,0.943738,0.791798,0.664624,0.704520,0.604786,0.559377,0.477323,...,0.411646,0.370676,0.317090,0.311332,0.288629,0.277215,0.280209,0.263937,0.244267,0.377503


,2020-04-27,2020-04-28,2020-04-29,2020-04-30,2020-05-01,2020-05-02,2020-05-03,2020-05-04,2020-05-05,2020-05-06,...,2020-05-08,2020-05-09,2020-05-10,2020-05-11,2020-05-12,2020-05-13,2020-05-14,2020-05-15,2020-05-16,2020-05-17
Measure,,,,,,,,,,,,,,,,,,,,,
RMSE,86.822423,136.157735,183.658719,205.175857,200.197824,212.222508,243.033487,276.612782,330.366901,367.467000,...,422.654100,440.816712,706.989575,758.915717,780.291446,785.177811,831.985573,849.784359,865.666259,867.591484
MAE,41.055865,63.288131,95.245762,107.585357,110.543198,120.252792,145.858271,161.238708,189.420883,206.108551,...,223.460450,231.832568,356.902104,377.851932,389.212727,396.129343,415.399316,425.450191,430.022978,435.728561
MAPE,0.004229,0.005695,0.007345,0.008976,0.010200,0.012119,0.014396,0.015275,0.019136,0.019797,...,0.021880,0.022403,0.024793,0.027032,0.027237,0.028476,0.029271,0.030619,0.031964,0.032723
RMSE_NAIVE,163.473964,295.270973,452.741881,581.883059,666.505380,785.816304,963.734704,1097.762725,1286.643634,1447.566179,...,1717.378342,1876.265468,1507.575293,1650.179831,1835.854440,2045.872366,2206.113754,2391.158825,2566.993347,2761.105957
MAE_NAIVE,60.052632,109.263158,170.157895,219.368421,251.736842,292.421053,346.789474,401.000000,470.473684,529.736842,...,634.263158,683.473684,611.684211,667.894737,732.684211,802.368421,857.052632,920.000000,969.473684,1029.473684
MAE_MASE,0.683665,0.579227,0.559749,0.490432,0.439122,0.411232,0.420596,0.402092,0.402617,0.389077,...,0.352315,0.339198,0.583474,0.565736,0.531215,0.493700,0.484684,0.462446,0.443563,0.423254
RMSE_MASE,0.531109,0.461128,0.405659,0.352607,0.300369,0.270066,0.252179,0.251979,0.256766,0.253852,...,0.246104,0.234944,0.468958,0.459899,0.425029,0.383786,0.377127,0.355386,0.337230,0.314219


,2020-05-02 00:00:00,2020-05-03 00:00:00,2020-05-04 00:00:00,2020-05-05 00:00:00,2020-05-06 00:00:00,2020-05-07 00:00:00,2020-05-08 00:00:00,2020-05-09 00:00:00,2020-05-10 00:00:00,2020-05-11 00:00:00,2020-05-12 00:00:00,2020-05-13 00:00:00,2020-05-14 00:00:00,2020-05-15 00:00:00,2020-05-16 00:00:00,2020-05-17 00:00:00
Measure,,,,,,,,,,,,,,,,
RMSE,24.061165,84.041975,99.535160,161.930430,188.838410,164.801577,198.317275,209.850458,432.519377,457.090760,451.523831,427.068582,453.546162,448.243741,440.168067,423.016142
MAE,15.468064,40.139475,51.352729,78.091188,95.789897,93.788175,111.986043,120.948072,181.270081,192.979791,195.659720,195.857076,210.044584,219.200464,218.820907,219.002789
MAPE,0.002520,0.005060,0.006105,0.009462,0.010309,0.010417,0.011377,0.011919,0.013041,0.015401,0.015241,0.016581,0.017122,0.018350,0.019727,0.020445
RMSE_NAIVE,40.638328,140.852667,180.742183,278.962929,351.111993,365.057386,446.630720,512.444862,231.357550,268.856350,325.603488,411.342347,471.721260,550.641588,622.023777,721.889881
MAE_NAIVE,19.157895,58.631579,82.473684,123.052632,153.631579,167.631579,200.157895,222.263158,133.473684,160.263158,200.263158,245.684211,276.842105,311.421053,333.894737,369.526316
MAE_MASE,0.807399,0.684605,0.622656,0.634616,0.623504,0.559490,0.559489,0.544166,1.358096,1.204143,0.977013,0.797190,0.758716,0.703872,0.655359,0.592658
RMSE_MASE,0.592081,0.596666,0.550702,0.580473,0.537830,0.451440,0.444030,0.409508,1.869485,1.700130,1.386729,1.038231,0.961471,0.814039,0.707639,0.585984


In [82]:
ps = plot_metric_for_dates(fs, dates_los_alamos, 'Los Alamos', 'los_alamos', 'MAE', 'Hawaii')
show(ps[0])

### Northeastern GLEAM

In [556]:
dates_gleam = [
    #'2020-04-12',
    #'2020-04-19', 
    '2020-04-27',
    #'2020-05-05',
]
p = plot_metric_for_dates(fs, dates_gleam, 'GLEAM', '../data/gleam/predictions_{}.csv', 'MAE')
show(p)
_ = export_png(p, filename='/tmp/gleam.png')

2020-04-28     21.156863
2020-04-29     43.372549
2020-04-30     61.980392
2020-05-01     80.078431
2020-05-02     95.921569
2020-05-03    106.549020
2020-05-04    113.156863
2020-05-05    136.843137
2020-05-06    145.705882
2020-05-07    167.647059
2020-05-08    186.156863
Name: MAE_NAIVE, dtype: float64
2020-04-28     14.625000
2020-04-29     28.500000
2020-04-30     38.062500
2020-05-01     52.395833
2020-05-02     65.833333
2020-05-03     76.791667
2020-05-04     85.645833
2020-05-05    100.583333
2020-05-06    103.770833
2020-05-07    120.291667
2020-05-08    137.416667
Name: MAE_NAIVE, dtype: float64


### MIT

In [573]:
dates_mit = [
    '2020-04-23',
    '2020-04-28', 
    '2020-05-01',
    #'2020-05-03',
]
p = plot_metric_for_dates(fs, dates_mit, 'MIT', '../data/mit/predictions_{}.csv', 'MAE')
show(p)
_ = export_png(p, filename='/tmp/mit.png')

2020-04-24    118.784314
2020-04-25    112.313725
2020-04-26     96.235294
2020-04-27     91.627451
2020-04-28     91.098039
2020-04-29     98.333333
2020-04-30    100.725490
2020-05-01     95.137255
2020-05-02     87.901961
2020-05-03     79.941176
2020-05-04     66.568627
2020-05-05     73.960784
2020-05-06     85.764706
2020-05-07     92.843137
2020-05-08    108.098039
Name: MAE, dtype: float64
2020-04-29    103.686275
2020-04-30    105.607843
2020-05-01    101.784314
2020-05-02     93.058824
2020-05-03     88.901961
2020-05-04     87.686275
2020-05-05     90.019608
2020-05-06    103.568627
2020-05-07    105.980392
2020-05-08    109.960784
Name: MAE, dtype: float64
2020-05-02    110.764706
2020-05-03    107.019608
2020-05-04    100.862745
2020-05-05    108.450980
2020-05-06    126.392157
2020-05-07    130.941176
2020-05-08    133.294118
Name: MAE, dtype: float64
